In [ ]:
import pickle5 as pickle
from data_lp import LPDataset,get_image_transform
from vocab import Vocabulary
import json
import torch
from tqdm import tqdm
import pdb


def default_collate_fn(data, caption_lim = 512):
  # Sort a data list by sentence length
    data.sort(key=lambda x: len(x[1]), reverse=True)
    images, spoken_target, ocr_target, index, img_ids = zip(*data)
    # Merge images (convert tuple of 3D tensor to 4D tensor)
    images = torch.stack(images, 0)

    cap_lengths = torch.tensor([len(cap) if len(cap) <= caption_lim else caption_lim for cap in spoken_target])
    spoken_output = torch.zeros(len(spoken_target), caption_lim).long()
    

    for i, cap in enumerate(spoken_target):
      end = cap_lengths[i]
      if end <= caption_lim:
        spoken_output[i, :end] = cap[:end]
      else:
        cap_lengths[i] = caption_lim
        spoken_output[i, :end] = cap[:caption_lim]
    return images, spoken_output, ocr_target, cap_lengths, index, img_ids



In [ ]:

#define speaker
sp = 'psy-1'

#define where data is downloaded - this is the zip file that you download 
root_data_dir = '/projects/dataset_processed/dongwonl/data/'
sp_data_dir = '{}/{}'.format(root_data_dir, sp)

#define where the training script with data_lp.py is in
src_dr = '/work/dongwonl/lecture_project/kumon'

#define where the vocab folder is in 
target_vocab_path = './vocab/%s_vocab.pkl' % sp
vocab = pickle.load(open(target_vocab_path, 'rb'))

#read jsons 
with open("{}/{}/{}_figs.json".format(root_data_dir, sp,sp), 'r') as f:
 fig_json = json.loads(f.read())

with open("{}/{}/{}.json".format(root_data_dir, sp,sp), 'r') as j:
 cap_json = json.loads(j.read())

with open("{}/{}/{}_capfig.json".format(root_data_dir, sp,sp), 'r') as c:
 connect_json = json.loads(c.read())

#load data
transform = get_image_transform()
wemb_type = 'bert'
dataset = LPDataset(cap_json, fig_json, connect_json, vocab, sp_data_dir, wemb_type, transform)
loader = torch.utils.data.DataLoader(dataset=dataset,
                                        batch_size=2,
                                        shuffle=True,
                                        pin_memory=True,
                                        num_workers=1,
                                        collate_fn = default_collate_fn
                                        )

In [ ]:
'''
Sample Loop 

images: normalized images of figures (3 x 224 x 224)
spoken_output: word tokens for spoken language (glove or bert)
ocr_target: word tokens for ocr on-slide text (glove or bert)
cap_lengths: length of captions
index: number image ids
img_ids: string image ids

'''

for itr, data in enumerate(tqdm(loader)):
    images, spoken_output, ocr_target, cap_lengths, index, img_ids = data
    
    